# Consigna del desafío 1

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [2]:
# 1. Cargar datos
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [5]:
# en el atributo `data` accedemos al texto
print(train.data[34])


Once again, it appears that the one-eyed man has appeared in the land of the sighted
and for some strange resaon has appointed himself the ruler and supreme power.


In [3]:
# en el atributo `data` accedemos al texto
print(train.data[30])


The front covers should be available from Sony.  Check with a local car
stereo shop.  You will probably (definitely) have to provide the units 
serial number and hopefully you had registered the warranty card.  I 
don't know the cost, but replacements have to be available to people
who damage the face cover, so it stands to reason that it can be replaced.

As to deterring theft:

When I worked for a stereo shop, we referred the customer to a Sony 800
number.  We would not sell the face, nor did we have them available.  Most
people who came in asking for the face cover (or a pullout sleave for that
matter) would look very disheartened to find that they acquired a deck
they couldn't use.  If theft occurs with these decks, notify Sony.  Serial
numbers do catch theives.


## 1. Vectorizar documentos. 
Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [6]:
# 2. Vectorización
vectorizer = TfidfVectorizer(stop_words='english', 
                             ngram_range=(1, 2), 
                             max_df=0.95, 
                             min_df=2)

X_train = vectorizer.fit_transform(train.data)
X_test = vectorizer.transform(test.data)
y_train = train.target
y_test = test.target

In [7]:
import random

# Estrategia de Resolucion 
El siguiente codigo realiza:
- Se seleccionan 5 documentos aleatoriamente del conjunto de entrenamiento para analizar.
- Para cada documento seleccionado, se calcula la similitud del coseno con todos los documentos del corpus.
- Se ordenan los documentos según su similitud y se seleccionan los 5 más similares (excluyendo el propio documento).
- Se imprimen las clases (temas) de los documentos más similares, para evaluar si tiene sentido semántico la similitud encontrada.

In [9]:
random.seed(42)
random_docs = random.sample(range(X_train.shape[0]), 5)
print("Análisis de documentos similares:")
for doc_idx in random_docs:
    sim = cosine_similarity(X_train[doc_idx], X_train).flatten()
    top_indices = sim.argsort()[::-1][1:6]
    print(f"\nDocumento base (clase: {train.target_names[y_train[doc_idx]]}):")
    for i in top_indices:
        print(f" - Similar a doc {i} (clase: {train.target_names[y_train[i]]})")

Análisis de documentos similares:

Documento base (clase: rec.sport.hockey):
 - Similar a doc 8985 (clase: rec.sport.baseball)
 - Similar a doc 5064 (clase: rec.sport.hockey)
 - Similar a doc 3769 (clase: rec.sport.baseball)
 - Similar a doc 1833 (clase: rec.sport.baseball)
 - Similar a doc 5640 (clase: sci.crypt)

Documento base (clase: comp.sys.mac.hardware):
 - Similar a doc 9921 (clase: comp.sys.mac.hardware)
 - Similar a doc 5509 (clase: comp.sys.mac.hardware)
 - Similar a doc 6364 (clase: comp.sys.mac.hardware)
 - Similar a doc 6772 (clase: comp.sys.mac.hardware)
 - Similar a doc 4359 (clase: comp.sys.mac.hardware)

Documento base (clase: comp.graphics):
 - Similar a doc 3444 (clase: comp.graphics)
 - Similar a doc 1764 (clase: comp.graphics)
 - Similar a doc 5905 (clase: comp.graphics)
 - Similar a doc 6117 (clase: comp.windows.x)
 - Similar a doc 5799 (clase: comp.graphics)

Documento base (clase: rec.autos):
 - Similar a doc 4211 (clase: rec.motorcycles)
 - Similar a doc 2971 

## 2. Entrenar modelos de clasificación Naïve Bayes.
Para maximizar el desempeño de clasificación usar (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

## Estrategia de resolucion:
El siguiente codigo realiza:
- Se definen los dos modelos a comparar: MultinomialNB y ComplementNB.
- Para cada modelo se iteran el entrenamiento, prediccion y calculo del score.

In [8]:
# 4. Modelos Naive Bayes
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    score = f1_score(y_test, preds, average='macro')
    print(f"{name} → Macro F1-score: {score:.4f}")

MultinomialNB → Macro F1-score: 0.6504
ComplementNB → Macro F1-score: 0.7037


Ambos modelos logran resultados decentes, pero ComplementNB supera claramente a MultinomialNB.

La mejora (~0.05) refleja que la estrategia de usar información de clases complementarias ayuda a generalizar mejor, especialmente en datasets con alta variedad temática y desbalance como 20 Newsgroups.

## 3. Transponer la matriz documento-término. 
De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Objetivo
Evaluar qué palabras del corpus son semánticamente similares en función del contexto en que aparecen. Esto se logra mediante:

Representar cada palabra como un vector (basado en los documentos donde aparece).

Calcular la similaridad del coseno entre esos vectores.

Este enfoque se basa en la idea del espacio vectorial de términos, donde dos palabras son similares si aparecen en contextos similares (principio de distribución semántica de Firth: "You shall know a word by the company it keeps.").



### Estrategia de resolucion
- Obtener el diccionario inverso: idx2word.
-  Transponer la matriz documento-término. La matriz X_train tiene forma (n_docs, n_terms) → cada fila es un documento. Al transponerla, obtenemos (n_terms, n_docs) → cada fila ahora representa una palabra como un vector en el espacio de documentos.
-  Calcular la matriz de similaridad entre términos. Se calcula la similaridad del coseno entre todos los vectores de palabras. El resultado es una matriz simétrica de forma (n_terms, n_terms), donde: term_sim[i, j] es la similaridad entre la palabra i y la palabra j.
-  Definir palabras relevantes. Se seleccionan manualmente 5 palabras de interés, evitando términos poco interpretables o muy raros. Esto es importante para poder analizar semánticamente los resultados.
-  Buscar palabras similares para cada palabra objetivo. Para cada palabra:
   -  Se obtiene su índice.
   -  Se accede a su fila de similaridades term_sim[idx].
   -  Se ordenan los índices de términos por mayor similaridad (argsort()[::-1]) y se toman los 5 más similares (excluyendo el propio término si fuera necesario).
   -  Se imprime la lista de términos más cercanos semánticamente.



In [11]:
idx2word = {v: k for k, v in vectorizer.vocabulary_.items()}
X_term_doc = X_train.T
term_sim = cosine_similarity(X_term_doc)
words = ['jesus', 'car', 'windows', 'science', 'game']
print("\nPalabras más similares:")
for word in words:
    idx = vectorizer.vocabulary_.get(word)
    if idx:
        sim_words = term_sim[idx].argsort()[::-1][1:6]
        print(f"'{word}' → {[idx2word[i] for i in sim_words]}")
    else:
        print(f"'{word}' no está en el vocabulario.")


Palabras más similares:
'jesus' → ['jesus said', 'jesus christ', 'christ', 'jesus did', 'god']
'car' → ['car car', 'new car', 'bought car', 'car dealer', 'buying car']
'windows' → ['ms windows', 'dos', 'windows nt', 'dos windows', 'windows windows']
'science' → ['collection competing', 'religious sects', 'like science', 'psychology like', 'competing religious']
'game' → ['game game', 'game espn', 'games', 'wings game', 'game want']
